In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
from tabulate import tabulate

In [2]:
base_model_name = "AmaanP314/youtube-xlm-roberta-base-sentiment-multilingual"
finetuned_model_name = "gajula21/youtube-sentiment-model-telugu"

In [3]:
base_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(base_model_name)

finetuned_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_name)
finetuned_model = AutoModelForSequenceClassification.from_pretrained(finetuned_model_name)

In [9]:
labels = {0: "Negative", 1: "Neutral", 2: "Positive"}

comments_telugu = [
    "చాలా మంచి వీడియో! ఇన్ఫర్మేటివ్ గా ఉంది.",  # Positive
    "సూపర్ కంటెంట్! ఇలాంటి వీడియోలు మరిన్ని చేయండి.",  # Positive
    "మీరు చెప్పిన విధానం చాలా బాగుంది, అర్థం చేసుకోవడానికి సులభంగా ఉంది.",  # Positive
    "నవ్వు తెప్పించింది ఈ వీడియో 😂",  # Positive
    "సంగీతం అద్భుతంగా ఉంది!",  # Positive
    "ధన్యవాదాలు.",  # Neutral
    "ఓకే వీడియో.",  # Neutral
    "నేను తర్వాత చూస్తాను.",  # Neutral
    "ఇది ఎలా పని చేస్తుంది?",  # Neutral
    "సమయం ఎంత పట్టింది?",  # Neutral
    "ఇది చాలా నిదానంగా ఉంది.",  # Negative
    "నాకు నచ్చలేదు.",  # Negative
    "సౌండ్ సరిగా లేదు.",  # Negative
    "చాలా తప్పులు ఉన్నాయి.",  # Negative
    "ఇది సమయం వృధా.",  # Negative
    "చెత్త వీడియో!",  # Negative
]

comments_english = [
    "Absolutely loved this! Thanks for sharing.",  # Positive
    "This is so helpful, I learned a lot.",  # Positive
    "The editing is fantastic!",  # Positive
    "Made my day! 😊",  # Positive
    "Awesome content, keep it up!",  # Positive
    "Thanks.",  # Neutral
    "Interesting video.",  # Neutral
    "I'll check this out later.",  # Neutral
    "How long did this take to make?",  # Neutral
    "What software did you use?",  # Neutral
    "This was boring.",  # Negative
    "The quality is really bad.",  # Negative
    "I didn't understand anything.",  # Negative
    "Too long and repetitive.",  # Negative
    "Don't waste your time watching this.",  # Negative
    "This is terrible!",  # Negative
    "Why is it so laggy?"  # Negative (implies a problem)
]

In [10]:
def predict_sentiment(model, tokenizer, texts, label_map):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    preds = torch.argmax(outputs.logits, dim=1)
    return [label_map[p.item()] for p in preds]

In [11]:
base_preds_telugu = predict_sentiment(base_model, base_tokenizer, comments_telugu, labels)
finetuned_preds_telugu = predict_sentiment(finetuned_model, finetuned_tokenizer, comments_telugu, labels)

base_preds_english = predict_sentiment(base_model, base_tokenizer, comments_english, labels)
finetuned_preds_english = predict_sentiment(finetuned_model, finetuned_tokenizer, comments_english, labels)

In [12]:
def build_comparison_df(comments, base_preds, finetuned_preds):
    return pd.DataFrame({
        "Comment": comments,
        "Base Model Prediction": base_preds,
        "Fine-Tuned Model Prediction": finetuned_preds
    })

df_telugu = build_comparison_df(comments_telugu, base_preds_telugu, finetuned_preds_telugu)
df_english = build_comparison_df(comments_english, base_preds_english, finetuned_preds_english)

In [ ]:
print("\nTelugu Comments Sentiment Comparison:\n")
print(tabulate(df_telugu, headers='keys', tablefmt='grid', showindex=False))

print("\nEnglish Comments Sentiment Comparison:\n")
print(tabulate(df_english, headers='keys', tablefmt='grid', showindex=False))


Telugu Comments Sentiment Comparison:

+------------------------------------------+-------------------------+-------------------------------+
| Comment                                  | Base Model Prediction   | Fine-Tuned Model Prediction   |
+==========================================+=========================+===============================+
| చాలా మంచి వీడియో! ఇన్ఫర్మేటివ్ గా ఉంది.                 | Positive                | Positive                      |
+------------------------------------------+-------------------------+-------------------------------+
| సూపర్ కంటెంట్! ఇలాంటి వీడియోలు మరిన్ని చేయండి.              | Positive                | Positive                      |
+------------------------------------------+-------------------------+-------------------------------+
| మీరు చెప్పిన విధానం చాలా బాగుంది, అర్థం చేసుకోవడానికి సులభంగా ఉంది. | Positive                | Positive                      |
+------------------------------------------+-------------------------+-----